<a href="https://colab.research.google.com/github/BaronVonBussin/Stuff/blob/main/_Transform_USConstitution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import tensorflow as tf

In [2]:
!pip install requests beautifulsoup4

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://usconstitution.net/const.txt'

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text,'html.parser')

text = soup.get_text()

print(text)

wrapped_text = [text]

Provided by USConstitution.net
------------------------------

[Note: Repealed text is not noted in this version.  Spelling errors have been
corrected in this version.  For an uncorrected, annotated version of the
Constitution, visit http://www.usconstitution.net/const.html ]

We the People of the United States, in Order to form a more perfect Union,
establish Justice, insure domestic Tranquility, provide for the common
defence, promote the general Welfare, and secure the Blessings of Liberty to
ourselves and our Posterity, do ordain and establish this Constitution for the
United States of America.

Article 1.

Section 1
All legislative Powers herein granted shall be vested in a Congress of the
United States, which shall consist of a Senate and House of Representatives.

Section 2
The House of Representatives shall be composed of Members chosen every second
Year by the People of the several States, and the Electors in each State shall
have the Qualifications requisite for Electors of t

In [4]:
from keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

#Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(wrapped_text)
total_words = len(tokenizer.word_index) + 1
print(total_words)

input_sequences = tokenizer.texts_to_sequences(wrapped_text)

input_sequences = pad_sequences(input_sequences, padding='post')
print(input_sequences[:1])

word_id = tokenizer.word_index["we"]
print(word_id)
#index_id = tokenizer.index_word(11)
print(tokenizer.index_word)

1182
[[ 124   11  400 ...    3   17 1181]]
402
{1: 'the', 2: 'of', 3: 'shall', 4: 'and', 5: 'to', 6: 'be', 7: 'or', 8: 'in', 9: 'states', 10: 'president', 11: 'by', 12: 'a', 13: 'for', 14: 'united', 15: 'state', 16: 'any', 17: 'have', 18: 'as', 19: 'congress', 20: 'such', 21: 'not', 22: 'may', 23: 'which', 24: 'no', 25: 'all', 26: 'from', 27: 'law', 28: 'on', 29: 'this', 30: 'office', 31: 'vice', 32: 'amendment', 33: 'person', 34: 'house', 35: 'but', 36: 'other', 37: 'representatives', 38: 'one', 39: 'article', 40: 'senate', 41: 'he', 42: 'their', 43: 'constitution', 44: 'each', 45: 'number', 46: 'that', 47: 'two', 48: 'if', 49: 'at', 50: 'section', 51: 'years', 52: 'thereof', 53: 'power', 54: 'time', 55: 'within', 56: 'they', 57: 'it', 58: 'an', 59: 'been', 60: '1', 61: '2', 62: 'his', 63: 'citizens', 64: 'under', 65: 'with', 66: 'nor', 67: 'several', 68: 'electors', 69: 'who', 70: 'when', 71: 'vote', 72: 'same', 73: 'duties', 74: 'persons', 75: 'them', 76: 'is', 77: 'during', 78: 'ma

In [5]:
from keras.models import Sequential, Model
from keras.layers import Input, Embedding

embedding_dim = 10

model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=input_sequences.shape[1]))

model.compile('adam', 'mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 7667, 10)          11820     
                                                                 
Total params: 11,820
Trainable params: 11,820
Non-trainable params: 0
_________________________________________________________________


In [6]:
sequences = tokenizer.texts_to_sequences(wrapped_text)
sequences_padded = pad_sequences(sequences)
input_layer = Input(shape=(None,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=10) (input_layer)
intermediate_model=Model(inputs=model.input, outputs=model.get_layer(index=0).output)
embeddings = intermediate_model.predict(sequences_padded)
print(embeddings[0][:30])

1/1 [==============================] - 0s 146ms/step
[[ 0.04849687 -0.01448889  0.043513   -0.00023944 -0.00027996 -0.03912339
  -0.00102498 -0.04446927  0.04978256  0.00052226]
 [ 0.03565067 -0.00538981  0.02897036  0.03710179  0.00502726  0.04696685
  -0.03071803 -0.0197101  -0.03496517 -0.0165927 ]
 [-0.02635088 -0.04961486 -0.02075619  0.02238784  0.04269947  0.03615326
   0.00342231 -0.00278974  0.01964964 -0.01824231]
 [ 0.02820945 -0.00128305 -0.01904122 -0.02237595 -0.0258196   0.02727229
   0.00828656 -0.03544328 -0.01655919 -0.01681892]
 [ 0.0042562   0.04807677 -0.00304264  0.0487091  -0.0453476  -0.04201653
  -0.00774341 -0.04185729  0.00778328 -0.03684492]
 [-0.01455544 -0.02305924 -0.00321128  0.02740678 -0.03402202  0.02361245
   0.03927309  0.02808667  0.02351487  0.01811388]
 [-0.03768629  0.01456782  0.03962084  0.00672557 -0.03278841 -0.0292811
  -0.02122976 -0.00534724  0.04412727 -0.04317329]
 [ 0.03190881  0.03233873  0.04729206  0.0243233  -0.00140411 -0.02980371

In [7]:
def positional_encoding(position, d_model):
  angle_rads = np.arange(position) [:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] //2)) / np.float32(d_model))

  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [8]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadSelfAttention,self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)
    k = self.wk(k)
    v = self.wv(v)

    q = self.split_heads(q, batch_size)
    k = self.split_heads(k, batch_size)
    v = self.split_heads(v, batch_size)

    scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
    output = self.dense(concat_attention)

    return output, attention_weights

  def scaled_dot_product_attention(self, q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    d_k = tf.cast(tf.shape(k)[-1],tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(d_k)

    if mask is not None:
      scaled_attention_logits += (mask * 1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)
    output = tf.matmul(attention_weights, v)

    return output, attention_weights

In [9]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadSelfAttention(d_model, num_heads)
    self.ffn = tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):
    attn_output, _ = self.mha(x, x, x,mask)
    attn_output = self.dropout(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)

    fnn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)

    return out2

#print(tf_cast)

In [10]:
#!pip install nltk

In [11]:
#from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer()
#wrapped_text = [text]
#X = vectorizer.fit_transform(wrapped_text)
#print(vectorizer.get_feature_names_out())
#print(X.toarray())

In [12]:
#print([t.text for t in doc])

In [13]:
#import nltk
#from nltk.tokenize import word_tokenize
#nltk.download('punkt')

In [ ]:
#nlp = spacy.load('en_core_web_sm')

In [ ]:
#doc = nlp(text)

In [ ]:
#import spacy

In [ ]:
#tokens = word_tokenize(text)
#print(tokens)

['Provided', 'by', 'USConstitution.net', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '[', 'Note', ':', 'Repealed', 'text', 'is', 'not', 'noted', 'in', 'this', 'version', '.', 'Spelling', 'errors', 'have', 'been', 'corrected', 'in', 'this', 'version', '.', 'For', 'an', 'uncorrected', ',', 'annotated', 'version', 'of', 'the', 'Constitution', ',', 'visit', 'http', ':', '//www.usconstitution.net/const.html', ']', 'We', 'the', 'People', 'of', 'the', 'United', 'States', ',', 'in', 'Order', 'to', 'form', 'a', 'more', 'perfect', 'Union', ',', 'establish', 'Justice', ',', 'insure', 'domestic', 'Tranquility', ',', 'provide', 'for', 'the', 'common', 'defence', ',', 'promote', 'the', 'general', 'Welfare', ',', 'and', 'secure', 'the', 'Blessings', 'of', 'Liberty', 'to', 'ourselves', 'and', 'our', 'Posterity', ',', 'do', 'ordain', 'and', 'establish', 'this', 'Constitution', 'for', 'the', 'United', 'States', 'of', 'America', '.', 'Article', '1', '.', 'Sec

In [ ]:
#!python -m spacy download en_core_web_sm

2023-08-16 08:05:07.396824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
